In [2]:
import pandas as pd
import os
from neo4j import GraphDatabase
from langchain_ollama import ChatOllama, OllamaEmbeddings
from dotenv import load_dotenv

load_dotenv()

# Настройки
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "password")

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
llm = ChatOllama(model="qwen3:8b", temperature=0)
embeddings = OllamaEmbeddings(
    model="qwen3-embedding:0.6b"
)

def load_initial_graph(csv_path):
    df = pd.read_csv(csv_path, sep=';')
    print(f"Загрузка {len(df)} компаний...")
    
    with driver.session() as session:
        # Создаем индексы
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (c:Company) REQUIRE c.ticker IS UNIQUE")
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (i:Industry) REQUIRE i.name IS UNIQUE")
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (s:Sector) REQUIRE s.name IS UNIQUE")
        
        # Импорт
        for _, row in df.iterrows():
            cypher = """
            MERGE (c:Company {ticker: $ticker})
            SET c.name = $name, c.description = $desc
            
            MERGE (i:Industry {name: $industry})
            MERGE (s:Sector {name: $sector})
            
            MERGE (c)-[:BELONGS_TO]->(i)
            MERGE (i)-[:PART_OF]->(s)
            """
            session.run(cypher, 
                        ticker=row['Ticker'], 
                        name=row['Name'], 
                        desc=row['Description'],
                        industry=row['Industry'],
                        sector=row['Sector'])
    print("✅ Граф построен.")

# 1. Запускаем загрузку (если еще не загружено)
# load_initial_graph('./data/sp500_graph_ready.csv')

c:\Users\ekurm\miniconda3\envs\kgraph\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def generate_industry_summaries():
    print("🚀 Начало генерации описаний для Индустрий...")
    
    with driver.session() as session:
        # 1. Получаем список всех индустрий
        industries = session.run("MATCH (i:Industry) RETURN i.name as name").value()
        
        for ind_name in industries:
            # 2. Берем описания всех компаний в этой индустрии
            result = session.run("""
                MATCH (c:Company)-[:BELONGS_TO]->(i:Industry {name: $name})
                RETURN c.name, c.description
            """, name=ind_name)
            
            companies_text = "\n".join([f"- {r['c.name']}: {r['c.description']}" for r in result])
            
            # 3. Промпт для саммаризации
            prompt = f"""
            Analyze the following companies in the '{ind_name}' industry:
            {companies_text[:10000]}  # Обрезаем, если слишком длинно

            Create a comprehensive summary (2-3 paragraphs) of this Industry. 
            Describe what these companies typically do, the technologies they use, 
            and the markets they serve. Do not list companies, synthesize the trends.
            """
            
            # 4. Генерация
            print(f"  generating summary for Industry: {ind_name}...")
            summary = llm.invoke(prompt).content
            
            # 5. Запись обратно в граф
            session.run("""
                MATCH (i:Industry {name: $name})
                SET i.description = $desc
            """, name=ind_name, desc=summary)

    print("✅ Индустрии описаны.")
def generate_sector_summaries():
    print("🚀 Начало генерации описаний для Секторов...")
    
    with driver.session() as session:
        sectors = session.run("MATCH (s:Sector) RETURN s.name as name").value()
        
        for sec_name in sectors:
            # Берем описания ИНДУСТРИЙ, входящих в сектор
            result = session.run("""
                MATCH (i:Industry)-[:PART_OF]->(s:Sector {name: $name})
                RETURN i.name, i.description
            """, name=sec_name)
            
            industries_text = "\n".join([f"- Industry {r['i.name']}: {r['i.description']}" for r in result])
            
            prompt = f"""
            Analyze the following industries within the '{sec_name}' sector:
            {industries_text}

            Create a high-level strategic summary of this Sector. 
            Explain the economic role of this sector, key drivers, and sub-verticals involved.
            """
            
            print(f"  generating summary for Sector: {sec_name}...")
            summary = llm.invoke(prompt).content
            
            session.run("""
                MATCH (s:Sector {name: $name})
                SET s.description = $desc
            """, name=sec_name, desc=summary)
            
    print("✅ Секторы описаны.")

In [ ]:
import os
from langchain_community.vectorstores import Neo4jVector
from dotenv import load_dotenv

load_dotenv()

# 1. Настройка модели эмбеддингов (должна совпадать с той, что будет при поиске)
print("⏳ Начинаю создание векторного индекса и вычисление эмбеддингов...")
print("Это может занять время, если компаний много...")

# 2. Эта команда делает две вещи:
#    а) Создает индекс с именем 'global_knowledge_index'
#    б) Проходит по всем узлам Company, берет их description, 
#       превращает в векторы и сохраняет в свойство 'embedding'
try:
    vector_store = Neo4jVector.from_existing_graph(
        embedding=embeddings,
        url=NEO4J_URI,
        username=NEO4J_USER,
        password=NEO4J_PASSWORD,
        index_name="global_knowledge_index",  # <--- Имя, на которое ругалась ошибка
        node_label="Company",
        text_node_properties=["description"], # Что векторизуем
        embedding_node_property="embedding",  # Куда сохраняем вектор
    )
    print("✅ Индекс 'global_knowledge_index' успешно создан!")
    
except Exception as e:
    print(f"❌ Ошибка: {e}")

⏳ Начинаю создание векторного индекса и вычисление эмбеддингов...
Это может занять время, если компаний много...
✅ Индекс 'global_knowledge_index' успешно создан!


In [ ]:
generate_industry_summaries()
generate_sector_summaries()    

In [3]:
from langchain_community.vectorstores import Neo4jVector

def graph_rag_search_unified(question):
    print(f"🔎 Вопрос: {question}")
    
    # 1. Превращаем вопрос пользователя в вектор
    query_vec = embeddings.embed_query(question)
    
    with driver.session() as session:
        # 2. Ищем в индексе 5 самых похожих узлов (неважно, Компания это, Сектор или Индустрия)
        cypher_query = """
        CALL db.index.vector.queryNodes('global_knowledge_index', 5, $embedding)
        YIELD node, score
        
        // Динамически определяем тип узла для красивого вывода
        RETURN 
            head(labels(node)) as type,
            node.name as name,
            node.description as description,
            score
        """
        
        results = session.run(cypher_query, embedding=query_vec)
        
        context_parts = []
        for r in results:
            # Формируем контекст для LLM
            context_parts.append(f"Type: {r['type']}\nName: {r['name']}\nInfo: {r['description']}\n")
            
        context_str = "\n---\n".join(context_parts)
        
        # 3. Отправляем в LLM
        prompt = f"""
        Based on the provided context, answer the user's question.
        If the context contains Industries or Sectors, use that high-level info to provide a broader answer.
        
        Context:
        {context_str}
        
        Question: {question}
        """
        
        response = llm.invoke(prompt)
        return response.content

print(graph_rag_search_unified("The Health Care sector Equipment companies"))

🔎 Вопрос: The Health Care sector Equipment companies
In the **Health Care sector**, **Equipment companies** primarily focus on manufacturing, distributing, or providing medical devices, diagnostic tools, and related technologies. Based on the context, the following companies are key players in this space:

---

### **1. GE HealthCare Technologies Inc.**  
- **Segments**: Imaging, Advanced Visualization Solutions (AVS), Patient Care Solutions (PCS), Pharmaceutical Diagnostics (PDx).  
- **Key Equipment/Products**:  
  - Imaging systems (CT, MRI, X-ray, ultrasound).  
  - Diagnostic agents (radiopharmaceuticals, contrast media).  
  - Patient monitoring devices, surgical tools, and digital solutions for treatment and monitoring.  
- **Focus**: Diagnostic and therapeutic equipment for hospitals, clinics, and research facilities.  

---

### **2. Cardinal Health, Inc.**  
- **Segments**: Pharmaceutical and Specialty Solutions, Global Medical Products and Distribution.  
- **Key Equipment/P

бенчмарк:
    Найти набор опорных вопросов и на основе них сделать свой бенчмарк или сгенерировать
    Выделить несколько типов вопросов
    ЕЩЁ генерировать правильные ответы 
